<img src="images/deep_neural_network.png" style="width:1024px;height:768px;">

In [3]:
import numpy as np
import copy

np.random.seed(1)

# Initialize

In [41]:
def initialize_parameters(layer_dims):
    """
    Wl - weight matrix of shape (layer_dims[l], layer_dims[l-1])\n
    bl - bias vector of shape (layer_dims[l], 1)
    :param layer_dims: array (list) containing the dimensions of each layer
    :return: dictionary containing the parameters of each layer: "W1", "b1", ..., "WL", "bL
    """
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        # Initialize all weights using "He" initialization to eliminate vanishing/exploding gradients
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2/layer_dims[l-1])
        # Initialize all biases to zero
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

In [43]:
np.random.seed(1)
t_parameters = initialize_parameters([5,4,3])
print(str(t_parameters["W1"]))
print(str(t_parameters["b1"]))
print(str(t_parameters["W2"]))
print(str(t_parameters["b2"]))
assert np.allclose(t_parameters["W1"], np.array([
    [1.02732621, -0.38690873, -0.33404515, -0.67860494, 0.54733184],
    [-1.45562088, 1.10351585, -0.48142952, 0.20177804,-0.15771567],
    [0.92471825, -1.30294739, -0.20391454, -0.2428973, 0.71705876],
    [-0.69563232, -0.10905317, -0.55520641, 0.02669832, 0.36860471]
]))
assert np.allclose(t_parameters["b1"], np.array(
    [0, 0, 0, 0]
))
assert np.allclose(t_parameters["W2"], np.array([
    [-0.77825528, 0.8094419, 0.63752091, 0.35531715],
    [ 0.63700135, -0.48346861, -0.08689651, -0.66168891],
    [-0.18942548, 0.37501795, -0.48907801, -0.28054711]
]))
assert np.allclose(t_parameters["b2"], np.array([[0.]]))

[[ 1.02732621 -0.38690873 -0.33404515 -0.67860494  0.54733184]
 [-1.45562088  1.10351585 -0.48142952  0.20177804 -0.15771567]
 [ 0.92471825 -1.30294739 -0.20391454 -0.2428973   0.71705876]
 [-0.69563232 -0.10905317 -0.55520641  0.02669832  0.36860471]]
[[0.]
 [0.]
 [0.]
 [0.]]
[[-0.77825528  0.8094419   0.63752091  0.35531715]
 [ 0.63700135 -0.48346861 -0.08689651 -0.66168891]
 [-0.18942548  0.37501795 -0.48907801 -0.28054711]]
[[0.]
 [0.]
 [0.]]


# Forward propagation

In [6]:
def sigmoid(Z):
    """
    Implements the sigmoid activation
    :param Z: numpy array of any shape
    :return: output of sigmoid(z), same shape as Z
    """
    A = 1/(1+np.exp(-Z))
    cache = Z
    return A, cache

In [7]:
def relu(Z):
    """
    Implement the RELU function
    :param Z: Output of the linear layer, of any shape
    :return:  output of relu(z), same shape as Z
    """
    A = np.maximum(0,Z)
    cache = Z
    return A, cache

In [8]:
def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.
    :param A: activations from previous layer (or input data): (size of previous layer, number of examples)
    :param W: weights matrix: numpy array of shape (size of current layer, size of previous layer)
    :param b: bias vector, numpy array of shape (size of the current layer, 1)
    :return:
        Z -- the input of the activation function, also called pre-activation parameter
        cache -- a python tuple containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    Z = W.dot(A) + b
    cache = (A, W, b)
    return Z, cache

In [9]:
np.random.seed(1)
t_A = np.random.randn(3,2)
t_W = np.random.randn(1,3)
t_b = np.random.randn(1,1)
t_Z, t_linear_cache = linear_forward(t_A, t_W, t_b)
assert np.allclose(t_Z, np.array([3.26295337, -1.23429987]))

In [10]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer
    :param A_prev: activations from previous layer (or input data): (size of previous layer, number of examples)
    :param W: weights matrix: numpy array of shape (size of current layer, size of previous layer)
    :param b: bias vector, numpy array of shape (size of the current layer, 1)
    :param activation: the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    :return:
        A -- the output of the activation function, also called the post-activation value
        cache -- a python tuple containing "linear_cache" and "activation_cache";
    """
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)

    cache = (linear_cache, activation_cache)
    return A, cache

In [11]:
np.random.seed(1)
t_A_prev = np.random.randn(3,2)
t_W = np.random.randn(1,3)
t_b = np.random.randn(1,1)
t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "sigmoid")
assert np.allclose(t_A, np.array([[0.96313579, 0.22542973]]));
t_A, t_linear_activation_cache = linear_activation_forward(t_A_prev, t_W, t_b, activation = "relu")
assert np.allclose(t_A, np.array([[3.26295337, 0.0]]));

In [12]:
def forward_propagate(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2 # Number of layers in the neural network

    # Implement [LINEAR->RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)

    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], activation = "sigmoid")
    caches.append(cache)

    return AL, caches

In [13]:
np.random.seed(1)
t_X = np.random.randn(5,4)
t_parameters = {
    "W1": np.random.randn(4,5),
    "b1": np.random.randn(4,1),
    "W2": np.random.randn(3,4),
    "b2": np.random.randn(3,1),
    "W3": np.random.randn(1,3),
    "b3": np.random.randn(1,1)
}
t_AL, t_caches = forward_propagate(t_X, t_parameters)
print("AL = " + str(t_AL))
assert np.allclose(t_AL, np.array([[0.77634609, 0.9998399, 0.99021857, 0.33755508]]))

AL = [[0.77634609 0.9998399  0.99021857 0.33755508]]


# Cost function

In [14]:
def compute_cost(AL, Y):
    """
    Implement the cost function
    :param AL: probability vector corresponding to label predictions, shape (1, number of examples)
    :param Y: true "label" vector
    :return: cross-entropy cost
    """
    m = Y.shape[1]
    #cost = np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL), axis=1) / -m
    cost = (1./m) * (-np.dot(Y,np.log(AL).T) - np.dot(1-Y, np.log(1-AL).T))
    cost = np.squeeze(cost)
    return cost

In [15]:
t_Y = np.asarray([[1, 1, 0]])
t_AL = np.array([[0.8, 0.9, 0.4]])
t_cost = compute_cost(t_AL, t_Y)
assert np.allclose(t_cost, 0.2797765635793422)

# Backward propagation

In [16]:
def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    return dZ

In [17]:
def sigmoid_backward(dA, cache):
    """
    Implement the backward propagation for a single SIGMOID unit.

    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently

    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    Z = cache
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    return dZ

In [18]:
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1./m * np.dot(dZ,A_prev.T)
    db = 1./m * np.sum(dZ, axis = 1, keepdims = True)
    dA_prev = np.dot(W.T,dZ)
    return dA_prev, dW, db

In [19]:
np.random.seed(1)
t_dZ = np.random.randn(3,4)
t_linear_cache = (np.random.randn(5,4), np.random.randn(3,5), np.random.randn(3,1))
t_dA_prev, t_dW, t_db = linear_backward(t_dZ, t_linear_cache)
assert np.allclose(t_dA_prev, np.array([
    [-1.15171336, 0.06718465, -0.3204696, 2.09812712],
    [ 0.60345879, -3.72508701, 5.81700741, -3.84326836],
    [-0.4319552, -1.30987417, 1.72354705, 0.05070578],
    [-0.38981415, 0.60811244, -1.25938424, 1.47191593],
    [-2.52214926, 2.67882552, -0.67947465, 1.48119548],
]))
assert np.allclose(t_dW, np.array([
    [0.07313866, -0.0976715, -0.87585828, 0.73763362, 0.00785716],
    [0.85508818, 0.37530413, -0.59912655, 0.71278189, -0.58931808],
    [0.97913304, -0.24376494, -0.08839671, 0.55151192, -0.10290907],
]))
assert np.allclose(t_db, np.array([
    [-0.14713786],
    [-0.11313155],
    [-0.13209101]
]))

In [20]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.

    Arguments:
    dA -- post-activation gradient for current layer l
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    return dA_prev, dW, db

In [22]:
np.random.seed(2)
t_dAL = np.random.randn(1,2)
t_A = np.random.randn(3,2)
t_W = np.random.randn(1,3)
t_b = np.random.randn(1,1)
t_Z = np.random.randn(1,2)
t_linear_activation_cache = ((t_A, t_W, t_b), t_Z)

t_dA_prev, t_dW, t_db = linear_activation_backward(t_dAL, t_linear_activation_cache, activation = "sigmoid")
assert np.allclose(t_dA_prev, np.array([
    [0.11017994, 0.01105339],
    [0.09466817, 0.00949723],
    [-0.05743092, -0.00576154]
]))
assert np.allclose(t_dW, np.array([
    [0.10266786, 0.09778551, -0.01968084]
]))
assert np.allclose(t_db, np.array([
    [-0.05729622]
]))

t_dA_prev, t_dW, t_db = linear_activation_backward(t_dAL, t_linear_activation_cache, activation = "relu")
assert np.allclose(t_dA_prev, np.array([
    [0.44090989, 0.0],
    [0.37883606, 0.0],
    [-0.2298228, 0.0]
]))
assert np.allclose(t_dW, np.array([
    [0.44513824, 0.37371418, -0.10478989]
]))
assert np.allclose(t_db, np.array([
    [-0.20837892]
]))

In [23]:
def backward_propagate(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group

    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (there are (L-1) or them, indexes from 0 to L-2)
                the cache of linear_activation_forward() with "sigmoid" (there is one, index L-1)

    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ...
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ...
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation = "sigmoid")

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

In [24]:
np.random.seed(3)

t_AL = np.random.randn(1, 2)
t_Y = np.array([[1, 0]])

t_A1 = np.random.randn(4, 2)
t_W1 = np.random.randn(3,4)
t_b1 = np.random.randn(3,1)
t_Z1 = np.random.randn(3,2)
t_linear_cache_activation1 = ((t_A1, t_W1, t_b1), t_Z1)

t_A2 = np.random.randn(3,2)
t_W2 = np.random.randn(1,3)
t_b2 = np.random.randn(1,1)
t_Z2 = np.random.randn(1,2)
t_linear_cache_activation2 = ((t_A2, t_W2, t_b2), t_Z2)

t_AL, t_Y_assess, t_caches = t_AL, t_Y, (t_linear_cache_activation1, t_linear_cache_activation2)
t_grads = backward_propagate(t_AL, t_Y_assess, t_caches)

assert np.allclose(t_grads['dA0'], np.array([
    [0.0, 0.52257901],
    [0.0, -0.3269206],
    [0.0, -0.32070404],
    [0.0, -0.74079187]
]))
assert np.allclose(t_grads['dA1'], np.array([
    [0.12913162, -0.44014127],
    [-0.14175655, 0.48317296],
    [0.01663708, -0.05670698]
]))
assert np.allclose(t_grads['dW1'], np.array([
    [0.41010002, 0.07807203, 0.13798444, 0.10502167],
    [0.0, 0.0, 0.0, 0.0],
    [0.05283652, 0.01005865, 0.01777766, 0.0135308 ],
]))
assert np.allclose(t_grads['dW2'], np.array([
    [-0.39202432, -0.13325855, -0.04601089]
]))
assert np.allclose(t_grads['db1'], np.array([
    [-0.22007063],
    [0.0],
    [-0.02835349]
]))
assert np.allclose(t_grads['db2'], np.array([
    [0.15187861]
]))

# Update parameters

In [25]:
def update_parameters(params, grads, learning_rate):
    """
    Update parameters using gradient descent

    Arguments:
    parameters -- python dictionary containing your parameters
    grads -- python dictionary containing your gradients, output of L_model_backward

    Returns:
    parameters -- python dictionary containing your updated parameters
                  parameters["W" + str(l)] = ...
                  parameters["b" + str(l)] = ...
    """
    parameters = copy.deepcopy(params)
    L = len(parameters) // 2
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [26]:
np.random.seed(2)
t_W1 = np.random.randn(3,4)
t_b1 = np.random.randn(3,1)
t_W2 = np.random.randn(1,3)
t_b2 = np.random.randn(1,1)
t_parameters = {"W1": t_W1,
                "b1": t_b1,
                "W2": t_W2,
                "b2": t_b2}

np.random.seed(3)
t_dW1 = np.random.randn(3,4)
t_db1 = np.random.randn(3,1)
t_dW2 = np.random.randn(1,3)
t_db2 = np.random.randn(1,1)
t_grads = {"dW1": t_dW1,
           "db1": t_db1,
           "dW2": t_dW2,
           "db2": t_db2}
t_parameters = update_parameters(t_parameters, t_grads, 0.1)

assert np.allclose(t_parameters["W1"], np.array([
    [-0.59562069, -0.09991781, -2.14584584, 1.82662008],
    [-1.76569676, -0.80627147, 0.51115557, -1.18258802],
    [-1.0535704, -0.86128581, 0.68284052, 2.20374577]
]))
assert np.allclose(t_parameters["b1"], np.array([
    [-0.04659241],
    [-1.28888275],
    [0.53405496]
]))
assert np.allclose(t_parameters["W2"], np.array([
    [-0.55569196, 0.0354055, 1.32964895]
]))
assert np.allclose(t_parameters["b2"], np.array([
    [-0.84610769]
]))

# Train L-layer model

In [27]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    np.random.seed(1)
    costs = []
    parameters = initialize_parameters(layers_dims)
    for i in range(0, num_iterations):
        AL, caches = forward_propagate(X, parameters)
        cost = compute_cost(AL, Y)
        grads = backward_propagate(AL, Y, caches)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and (i % 100 == 0 or i == num_iterations - 1):
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0:
            costs.append(cost)
    return parameters, costs

## Load data

In [28]:
import h5py
from common import CV_DATA_DIR

In [29]:
def load_data():
    train_dataset = h5py.File(CV_DATA_DIR / "playground" / "dnn" / "train_catvnoncat.h5", "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File(CV_DATA_DIR / "playground" / "dnn" / "test_catvnoncat.h5", "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

In [30]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [31]:
print ("Number of training examples:", train_x_orig.shape[0])
print ("Number of testing examples:", test_x_orig.shape[0])
print ("Each image is of size: (" + str(train_x_orig.shape[1]) + ", " + str(train_x_orig.shape[1]) + ", 3)")

assert train_x_orig.shape == (209, 64, 64, 3)
assert train_y.shape == (1, 209)
assert test_x_orig.shape == (50, 64, 64, 3)
assert test_y.shape == (1, 50)

Number of training examples: 209
Number of testing examples: 50
Each image is of size: (64, 64, 3)


In [32]:
# Reshape the training and test examples
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

train_x's shape: (12288, 209)
test_x's shape: (12288, 50)


## Train

In [33]:
layers_dims = [12288, 20, 7, 5, 1] #  4-layer model

In [40]:
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 1, print_cost = False)
print("Cost after first iteration: " + str(costs[0]))

Cost after first iteration: 1.208125010600323


In [35]:
parameters, costs = L_layer_model(train_x, train_y, layers_dims, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 1.208125010600323
Cost after iteration 100: 0.6740577413973853
Cost after iteration 200: 0.6625049864829947
Cost after iteration 300: 0.655233447204667
Cost after iteration 400: 0.6504891112622437
Cost after iteration 500: 0.6473194475269372
Cost after iteration 600: 0.6449422527912865
Cost after iteration 700: 0.6430979384805492
Cost after iteration 800: 0.6414454339232503
Cost after iteration 900: 0.6398930793986607
Cost after iteration 1000: 0.6374058448197395
Cost after iteration 1100: 0.6353225187839247
Cost after iteration 1200: 0.6334206995910424
Cost after iteration 1300: 0.6309078606523231
Cost after iteration 1400: 0.6278703533430822
Cost after iteration 1500: 0.6243486119913267
Cost after iteration 1600: 0.6206564914287053
Cost after iteration 1700: 0.6165605693183634
Cost after iteration 1800: 0.5949506456607018
Cost after iteration 1900: 0.5707369311105821
Cost after iteration 2000: 0.5391408405511141
Cost after iteration 2100: 0.4834937236199692
Co

# Test

In [36]:
def predict(X, y, parameters):
    """
    This function is used to predict the results of a  L-layer neural network.

    Arguments:
    X -- data set of examples you would like to label
    parameters -- parameters of the trained model

    Returns:
    p -- predictions for the given dataset X
    """
    m = X.shape[1]
    p = np.zeros((1,m))

    probas, caches = forward_propagate(X, parameters)
    for i in range(0, probas.shape[1]):
        if probas[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    accuracy = np.sum((p == y) / m)
    return p, accuracy

In [38]:
pred_train, accuracy = predict(train_x, train_y, parameters)
print("Accuracy:", accuracy)
assert np.greater(accuracy, 0.9)

Accuracy: 0.9186602870813396
